# Аугментация данных
Ранее в рамках проекта нами была обучена модель для классификации текстов благотворительных сборов. Модель получает текст и должна предсказывать его тему: например, лечение детей, помощь животным и так далее. Модель в целом показывает удовлетворительные результаты, учитывая сложность задачи из-за большой лексической близости текстов, маленького объема данных и их несбалансированность. В этой части исследования попробуем улучшить показатели за счет методов компенсации несбалансированности данных.

### Импорт библиотек

In [3]:
pip install imblearn

     |████████████████████████████████| 167 kB 758 kB/s eta 0:00:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [109]:
from sklearn.model_selection import train_test_split, KFold 
# можно импортировать просто model_selection целиком, тогда при использ. сплита нужно писать ... = model_selection.train_test_split*
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score
from sklearn.linear_model import LogisticRegression 
# можно импортировать linear_model целиком, тогда дальше нужно писать ... = linear_model.LogisticRegression()*

# *это два разных способа импортировать эту функцию

In [4]:
# mbalanced-learn is an open-source python toolbox aiming at providing a wide range of methods to cope with the problem of imbalanced dataset

from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler

In [12]:
import pandas as pd
import pymorphy2
morph_analyzer = pymorphy2.MorphAnalyzer()
from nltk.stem.snowball import RussianStemmer
from nltk.corpus import stopwords
from string import digits
import re

### Импорт данных

In [11]:
df = pd.read_csv('/Users/liza/PycharmProjects/Planeta_project/plset_ver_010.csv')

In [129]:
# Уникальные названия категорий в датасете

for topic in np.unique(df['Category']):
                print(topic)

активизм_просвещение_профилактика
бездомные_кризис
взрослые_лечение_реабилитация
дети_лечение_реабилитация
животные
заключенные
малоимущие_бедность
мечты_подарки_праздники
наука_история_культура
паллиатив_уход
пожилые_ветераны
развитие_нко_инфраструктура
семейный_кризис
сироты_дети_из_неблагополучных семей
социализация_возможности
экология


In [121]:
# Смотрим соотношение данных в датасете, всего 2038 экземляров, очень неравномерное распределение








for k in np.unique(df['Category']):
    volume = len(df[df['Category']==k])
    volume_percent = round(volume*100/2038)
    print(f'{volume} = {volume_percent}%   {k}')

85 = 4%   активизм_просвещение_профилактика
37 = 2%   бездомные_кризис
71 = 3%   взрослые_лечение_реабилитация
535 = 26%   дети_лечение_реабилитация
245 = 12%   животные
6 = 0%   заключенные
66 = 3%   малоимущие_бедность
92 = 5%   мечты_подарки_праздники
10 = 0%   наука_история_культура
57 = 3%   паллиатив_уход
44 = 2%   пожилые_ветераны
84 = 4%   развитие_нко_инфраструктура
28 = 1%   семейный_кризис
262 = 13%   сироты_дети_из_неблагополучных семей
395 = 19%   социализация_возможности
21 = 1%   экология


### Baseline
На предыдущем этапе мы попробовали разные классификаторы и гиперапаметры. Лучшие результаты показала LogisticRegression без доп. параметров, за исключением max_iter, т.к. при значении по умолчанию модель просто не справляется с подсчетами. Если несколько раз перезапустить split (без random_state) и обучение, оценки варьируются от запуска к запуску. **Примерные значения, которые принимаем как baseline:**

accuracy: 0.78\
macro avg: 0.60\
weighted avg: 0.77 

+\\-0.04 в каждой метрике

**Повторим процесс обучения**

**Заготовка функций**

In [131]:
stop_words = stopwords.words('russian')
stop_words.extend(['это', '–', '-', 'фонд', 'наш', 'помощь', 'помогать',
                   'помочь', 'поддержать', 'поддержка', 'средство', 'который', 'весь',
                   'благотворительный', 'пожертвовать', 'пожертвование', 'деньги', 'рубль', 'год', 'день', 'тысяча',
                   'ваш', 'сегодня', 'завтра', 'этот', 'дать', 'проект', 'свой' ])

In [132]:
# функция для предобработки текстов

def prep(text):
    clean_text = text.translate(str.maketrans('', '', '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~«»№!—'))
    clean_text = clean_text.translate(str.maketrans('', '', digits)) # удаляем цифры
    clean_text = re.sub("-", " ", clean_text) # меняем тире на пробелы, т.к. maketrans их почему-то не выхватывает
    #clean_text = re.sub("[a-zA-Z]", "", clean_text)  # исключаем слова латиницей
    clean_text = clean_text.lower() # приводим все к нижнему регистру
    clean_text = clean_text.split() # разбиваем (токенизируем) по словам
    
    # можно возвращать слова без лемматизации\стемминга
    #words = [word for word in clean_text if word not in stop_words]
    #return words
    
    lemmas = [morph_analyzer.parse(word)[0].normal_form for word in clean_text]
    lemmas = [word for word in lemmas if word not in stop_words]
    return lemmas
    
    #ниже - альтернатива лемматицации, можно их переключать (за/раскомменчивать)
    #stemmer = RussianStemmer()
    #stemmed_words = [stemmer.stem(word) for word in clean_text]
    #return stemmed_words

In [161]:
# словарь для параметра class_weight для Лог. регр.

topic_weights = {
     'активизм_просвещение_профилактика': 0.08,
     'бездомные_кризис': 0.08,
     'взрослые_лечение_реабилитация': 0.08,
     'дети_лечение_реабилитация': 0.01,
     'животные': 0.01,
     'заключенные': 0.08,
     'малоимущие_бедность': 0.08,
     'мечты_подарки_праздники': 0.08,
     'наука_история_культура': 0.08,
     'паллиатив_уход': 0.08,
     'пожилые_ветераны': 0.08,
     'развитие_нко_инфраструктура': 0.08,
     'семейный_кризис': 0.08,
     'сироты_дети_из_неблагополучных семей': 0.01,
     'социализация_возможности': 0.01,
     'экология': 0.08}

**Получение признаков**\
При прошлых попытках обучения CountVectorizer показал себя лучше TfidfVectorizer, поэтому оставляем его

In [85]:
%%time

vec = CountVectorizer(tokenizer=prep) 
bag_of_words = vec.fit_transform(df.Description)

CPU times: user 3min 20s, sys: 1.09 s, total: 3min 22s
Wall time: 3min 26s


**Деление на выборки**

In [177]:
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.Category, stratify = df.Category, random_state=42) #, test_size=0.15)

#stratification means that the train_test_split method returns training and test subsets that have the same proportions of class labels as the input dataset.

**Обучение**

In [162]:
lr = LogisticRegression(max_iter=5000, class_weight=topic_weights) # опционально (почти не влияет на рез.): C=0.02, class_weight=topic_weights
clf = lr.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test), zero_division=0))

                                      precision    recall  f1-score   support

   активизм_просвещение_профилактика       0.65      0.62      0.63        21
                    бездомные_кризис       0.80      0.89      0.84         9
       взрослые_лечение_реабилитация       0.41      0.39      0.40        18
           дети_лечение_реабилитация       0.89      0.81      0.85       134
                            животные       0.97      0.98      0.98        61
                         заключенные       0.00      0.00      0.00         1
                 малоимущие_бедность       0.53      0.59      0.56        17
             мечты_подарки_праздники       0.68      0.65      0.67        23
              наука_история_культура       0.00      0.00      0.00         3
                      паллиатив_уход       0.86      0.86      0.86        14
                    пожилые_ветераны       0.69      0.82      0.75        11
         развитие_нко_инфраструктура       0.27      0.33      

⬆︎⬆︎⬆︎ Выше мы попробовали еще несколько параметров, которые не настраивали раньше
1. Для компенсации небольшого количества данных можно **изменять пропорции разделения на train и test (параметр test_size в split)**, чтобы увеличить объем данных для обучения. 

2. Также можно **в LogisticRegression в class_weight подать словарь**, который будет задавать веса классов. Class_weight, default: none, all classes are supposed to have weight one. The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y)). With a dictionary it penalizes mistakes in samples of class with class_weight. So higher class-weight means we want to put more emphasis on a class. If say class 0 is 19 times more frequent than class 1, we should increase the class_weight of class 1 relative to class 0, say {0:.1, 1:.9}.
These weights will be multiplied with sample_weight (passed through the fit method) if sample_weight is specified (sample_weight: Array of weights that are assigned to individual samples. If not provided, then each sample is given unit weight).\
Словарь задан выше. Параметр "включается" раскомментированием в строке. 

Итог:\
Стойкого улучшения ни то, ни другое не дает, метрики всегда варьируются в пределах известного периода и скорее зависят от конкретного случая разбиения. 




### Under-sampling и over-sampling

Для работы с несбалансированными данными нужно увеличить количество одних примеров или уменьшить количество других. Для этого существуют различные техники аугментации (увеличения, усиления) данных. В Python для этих целей есть библиотека imblearn (imbalanced-learn). Инструменты для аугментации: under-sampling, over-sampling и их комбинация.
 
**Under-sampling** уравновешивает данные за счет уменьшения размера превалирующего класса. Этот метод разумно использовать, когда количество данных достаточно велико, иначе есть риск остаться и вовсе без обучающих примеров.

**Over-sampling** применяется, когда данных недостаточно или количество экземпляров в миноритарном классе очень мало. При применении этой техники балансировка данных происходит за счет увеличения количества экземпляров в миноритарном классе. Новые элементы генерируются за счет: повторения, бутстрэппинга, SMOTE (Synthetic Minority Over-Sampling Technique) или ADASYN (Adaptive synthetic sampling).

[Источник](https://github.com/PragmaticsLab/NLP-course-AMI/blob/dev/seminars/sem3_classification.ipynb)

В нашем случае данных мало, поэтому будем использовать over-sampling.

In [125]:
# функция для обучения модели

def train_model(classifier, feature_vector_train, label, feature_vector_valid): # получает метод классификации, учебный вектор признаков, правильные лейблы к нему, проверочный вектор признаков
    classifier.fit(feature_vector_train, label) # fit запоминает вектор конкретного текста (документа) + даем соответствующие каждому лейблы
    predictions = classifier.predict(feature_vector_valid) # просим предсказать лейблы для текстов из тестовой выборки
    #return f1_score(y_test, predictions, average='weighted', zero_division=0) # возвращает оценку после соотнесения предсказанных лейблов с правильными из тестовой выборки
    return classification_report(y_test, predictions, zero_division=0) # можно переключать вид оценки (полный или только f-score или любой другой из sklearn metrics)

**Примечания к функции обучения** ⬆︎

* Точность в качестве метрики работает хорошо только на сбалансированных наборах данных, поэтому для оценки результатов работы алгоритма будем использовать F-меру. F-мера (F1 score) представляет собой совместную оценку точности и полноты. Вычисляется по формуле: F-мера = 2 * Точность * Полнота / (Точность + Полнота). Should be used **to compare classifier models**, not global accuracy.

* Выбор вывода оценки. С помощью комментирования # можно переключаться межу полным отчетом и только f1-score. Для f1-score можно менять тип average (по умолчанию это binary, но он не работает на мультиклассовом обучении):\
    **'micro':**
    Calculate metrics globally by counting the total true positives, false negatives and false positives.\
    **'macro':**
    Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.\
    **'weighted':**
    Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.\
    **'samples':**
    Calculate metrics for each instance, and find their average (only meaningful for multilabel classification where this differs from accuracy_score).

* Zero_division. Sets the value to return when there is a zero division. If set to “warn”, this acts as 0, but warnings are also raised. If we request an **average** of the score, we must take into account that a score of 0 may be included in the calculation and scikit-learn will show a warning.

\
**RandomOverSampler**\
Случайным образом дублируются некоторые элементы из миноритарного класса.

In [127]:
%%time
ros = RandomOverSampler(random_state=777)
ros_xtrain, ros_ytrain = ros.fit_sample(X_train, y_train) #пропускаем тренировочные тексты и лейблы через ROS
accuracyROS = train_model(LogisticRegression(max_iter=5000, random_state=0, class_weight=topic_weights),ros_xtrain, ros_ytrain, X_test) # вызываем функцию
print ("Logistic regression ROS:\n\n", accuracyROS) # выводим оценку, заложенную в функции train_model

Logistic regression ROS:

                                       precision    recall  f1-score   support

   активизм_просвещение_профилактика       0.33      0.15      0.21        13
                    бездомные_кризис       0.83      1.00      0.91         5
       взрослые_лечение_реабилитация       0.80      0.36      0.50        11
           дети_лечение_реабилитация       0.84      0.89      0.86        80
                            животные       0.90      1.00      0.95        37
                         заключенные       0.00      0.00      0.00         1
                 малоимущие_бедность       0.62      0.80      0.70        10
             мечты_подарки_праздники       0.69      0.79      0.73        14
              наука_история_культура       0.00      0.00      0.00         1
                      паллиатив_уход       1.00      0.89      0.94         9
                    пожилые_ветераны       1.00      0.57      0.73         7
         развитие_нко_инфраструктура

\
**SMOTE**\
Алгоритм SMOTE основан на идее генерации некоторого количества искусственных примеров, которые были бы «похожи» на имеющиеся в миноритарном классе, но при этом не дублировали их.

In [191]:
%%time

sm = SMOTE(random_state=777, k_neighbors=3)
sm_xtrain, sm_ytrain = sm.fit_sample(X_train, y_train)
accuracySMOTE = train_model(LogisticRegression(max_iter=5000, random_state=0), sm_xtrain, sm_ytrain, X_test)
print ("Logistic regression SMOTE:\n\n", accuracySMOTE)

Logistic regression SMOTE:

                                       precision    recall  f1-score   support

   активизм_просвещение_профилактика       0.50      0.52      0.51        21
                    бездомные_кризис       0.67      0.89      0.76         9
       взрослые_лечение_реабилитация       0.40      0.33      0.36        18
           дети_лечение_реабилитация       0.85      0.83      0.84       134
                            животные       0.97      0.98      0.98        61
                         заключенные       0.00      0.00      0.00         1
                 малоимущие_бедность       0.69      0.53      0.60        17
             мечты_подарки_праздники       0.86      0.52      0.65        23
              наука_история_культура       0.00      0.00      0.00         3
                      паллиатив_уход       0.92      0.86      0.89        14
                    пожилые_ветераны       0.71      0.91      0.80        11
         развитие_нко_инфраструкту

\
⬆︎⬆︎⬆︎ SMOTE may raise ValueError: "Expected n_neighbors <= n_samples, but n_samples = 5, n_neighbors = 6.\
n_neighbors (parameter k_neighbors, default = 5) is the number of nearest neighbours to be used to construct synthetic samples. 
Суть проблемы в том, что в обучающей выборке в каком-то из классов оказывается меньше экземпляров, чем нужно для формирования новых синтетических экземпляров в соответствии с параметром k_neighbors. 

A few solutions for the problem:

1. Calculate the minimum number of samples (n_samples) among the classes and select n_neighbors parameter of SMOTE class less or equal to n_samples. 
2. Use RandomOverSampler class which does not have a similar restriction.

⬇︎⬇︎⬇︎ Еще одна настройка SMOTE - это sampling_strategy: в каких пропорциях создаются новые синтетические экз. классов. У класса SMOTE есть несколько встроенных определений, а можно подать словарь. When dict, the keys correspond to the targeted classes. The values correspond to the desired number of samples for each targeted class.

Попробуем задать свой словарь. Желаемое количество экз. не должно быть меньше изначального количества в обучающей выборке. 

In [179]:
strategy_dict = {
     'активизм_просвещение_профилактика': 160,
     'бездомные_кризис': 160,
     'взрослые_лечение_реабилитация': 160,
     'дети_лечение_реабилитация': 401,
     'животные': 200,
     'заключенные': 160,
     'малоимущие_бедность': 160,
     'мечты_подарки_праздники': 160,
     'наука_история_культура': 160,
     'паллиатив_уход': 160,
     'пожилые_ветераны': 160,
     'развитие_нко_инфраструктура': 160,
     'семейный_кризис': 160,
     'сироты_дети_из_неблагополучных семей': 200,
     'социализация_возможности': 300,
     'экология': 160}

In [180]:
%%time

sm = SMOTE(sampling_strategy=strategy_dict, random_state=777, k_neighbors=4)
sm_xtrain, sm_ytrain = sm.fit_sample(X_train, y_train)
accuracySMOTE = train_model(LogisticRegression(max_iter=5000, random_state=0), sm_xtrain, sm_ytrain, X_test)
print ("Logistic regression SMOTE:\n\n", accuracySMOTE)

Logistic regression SMOTE:

                                       precision    recall  f1-score   support

   активизм_просвещение_профилактика       0.59      0.48      0.53        21
                    бездомные_кризис       0.67      0.89      0.76         9
       взрослые_лечение_реабилитация       0.40      0.33      0.36        18
           дети_лечение_реабилитация       0.88      0.84      0.85       134
                            животные       0.95      0.97      0.96        61
                         заключенные       0.00      0.00      0.00         1
                 малоимущие_бедность       0.64      0.53      0.58        17
             мечты_подарки_праздники       0.81      0.57      0.67        23
              наука_история_культура       0.00      0.00      0.00         3
                      паллиатив_уход       0.86      0.86      0.86        14
                    пожилые_ветераны       0.64      0.82      0.72        11
         развитие_нко_инфраструкту

\
Результат: разницы нет. 

\
**ADASYN или ASMO**\
Adaptive synthetic minority oversampling.
Сгенерировать искусственные записи в пределах отдельных кластеров на основе всех классов. Для каждого примера миноритарного класса находят m ближайших соседей, и на основе них (также как в SMOTE) создаются новые записи.


In [183]:
%%time

ad = ADASYN(random_state=777, n_neighbors=4)
ad_xtrain, ad_ytrain = ad.fit_sample(X_train, y_train)
accuracyADASYN = train_model(LogisticRegression(max_iter=5000, random_state=0), ad_xtrain, ad_ytrain, X_test)
print ("Logistic regression ADASYN:\n\n", accuracyADASYN)

Logistic regression ADASYN:

                                       precision    recall  f1-score   support

   активизм_просвещение_профилактика       0.52      0.57      0.55        21
                    бездомные_кризис       0.67      0.89      0.76         9
       взрослые_лечение_реабилитация       0.33      0.33      0.33        18
           дети_лечение_реабилитация       0.86      0.82      0.84       134
                            животные       0.95      1.00      0.98        61
                         заключенные       0.00      0.00      0.00         1
                 малоимущие_бедность       0.75      0.53      0.62        17
             мечты_подарки_праздники       0.77      0.43      0.56        23
              наука_история_культура       0.00      0.00      0.00         3
                      паллиатив_уход       0.86      0.86      0.86        14
                    пожилые_ветераны       0.64      0.82      0.72        11
         развитие_нко_инфраструкт

\
Результат: разницы нет. 

### Альтернативный способ кросс-валидации

Кросс-валидация - это когда мы делим датасет на подсеты для train и test. train_test.split - один из типов кросс-валидации в sklearn. 
Еще один - **k-fold cross-validation**.

Насколько я поняла, **это метод помогает не лучше обучить модель, а точнее оценить на небольшом датасете ее способности после обучения, т.к. мы несколько раз перетряхиваем данные и повторяем оценку.** 

Из википедии:

Illustration of k-fold cross-validation when n = 12 observations and k = 3. After data is shuffled, a total of 3 models will be trained and tested.
In k-fold cross-validation, the original sample is randomly partitioned into k equal sized subsamples. Of the k subsamples, a single subsample is retained as the validation data for testing the model, and the remaining k − 1 subsamples are used as training data. The cross-validation process is then repeated k times, with each of the k subsamples used exactly once as the validation data. The k results can then be averaged to produce a single estimation. The advantage of this method over repeated random sub-sampling (see below) is that all observations are used for both training and validation, and each observation is used for validation exactly once. 10-fold cross-validation is commonly used, but in general k remains an unfixed parameter.

For example, setting k = 2 results in 2-fold cross-validation. In 2-fold cross-validation, we randomly shuffle the dataset into two sets d0 and d1, so that both sets are equal size (this is usually implemented by shuffling the data array and then splitting it in two). We then train on d0 and validate on d1, followed by training on d1 and validating on d0.

When k = n (the number of observations), k-fold cross-validation is equivalent to leave-one-out cross-validation.

In stratified k-fold cross-validation, the partitions are selected so that the mean response value is approximately equal in all the partitions. In the case of binary classification, this means that each partition contains roughly the same proportions of the two types of class labels.

In repeated cross-validation the data is randomly split into k partitions several times. The performance of the model can thereby be averaged over several runs, but this is rarely desirable in practice.

***
It is a popular method because **it generally results in a less biased or less optimistic estimate of the model skill than other methods**, such as a simple train/test split.

In [189]:
# функция для обучения модели, копия функции, описанной выше с изменениями: return f1_score(kf_y_test.....)

def train_model_2(classifier, feature_vector_train, label, feature_vector_valid): # получает метод классификации, учебный вектор признаков, правильные лейблы к нему, проверочный вектор признаков
    classifier.fit(feature_vector_train, label) # fit запоминает вектор конкретного текста (документа) + даем соответсвующие им лейблы
    predictions = classifier.predict(feature_vector_valid) # просим теперь предсказать лейблы для текстов из тестовой выборки
    return f1_score(kf_y_test, predictions, average='weighted', zero_division=0) # возвращает оценку после соотнесения предсказанных лейблов с правильными из тестовой выборки

In [192]:
%%time 

# KFold
ros = RandomOverSampler(random_state=777)

X = bag_of_words # векторизованные тексты
y = df.Category # лейблы из датасета
kf = KFold(n_splits=15) # вызов KFold, количество сплитов можно менять
kf.get_n_splits(X)  # разбиваем
#print(kf)
for train_index, test_index in kf.split(X): # учим и оцениваем
    #print("TRAIN:", train_index, "TEST:", test_index)
    kf_X_train, kf_X_test = X[train_index], X[test_index]
    kf_y_train, kf_y_test = y[train_index], y[test_index]
    kf_ros_xtrain, kf_ros_ytrain = ros.fit_sample(kf_X_train, kf_y_train)
    kf_accuracyROS = train_model_2(LogisticRegression(max_iter=5000, random_state=0),kf_ros_xtrain, kf_ros_ytrain, kf_X_test)
    print(kf_accuracyROS)

0.7213958534233366
0.8329895594601476
0.8461618701482438
0.7106169518265688
0.8902255104940527
0.7822482198106386
0.7657016256886547
0.7804311572500428
0.7967710157680223
0.736866820306718
0.770622004862818
0.67765909873203
0.7509785089770742
0.6959227811590519
0.5935457957680179
CPU times: user 27min 48s, sys: 2min 50s, total: 30min 38s
Wall time: 10min 37s


In [195]:
%%time 

# Пробуем то же самое без ROS

X = bag_of_words # векторизованные тексты
y = df.Category # лейблы из датасета
kf = KFold(n_splits=15) # вызов KFold, количество сплитов можно менять
kf.get_n_splits(X)  # разбиваем
#print(kf)
for train_index, test_index in kf.split(X): # учим и оцениваем
    #print("TRAIN:", train_index, "TEST:", test_index)
    kf_X_train, kf_X_test = X[train_index], X[test_index]
    kf_y_train, kf_y_test = y[train_index], y[test_index]
    kf_accuracy = train_model_2(LogisticRegression(max_iter=5000, random_state=0),kf_X_train, kf_y_train, kf_X_test)
    print(kf_accuracy)

0.7302526251017173
0.8273778167983612
0.8287180255217856
0.6985739750445632
0.8754106187929719
0.745771633358549
0.7648602268050531
0.7733937117781529
0.7829275681903968
0.7035525228995968
0.7740292652956718
0.6594350277697194
0.719034527150685
0.6897324170400265
0.5854170021258628
CPU times: user 18min 2s, sys: 2min, total: 20min 3s
Wall time: 7min 5s


Результат: При 15 раундах weighted f1-score может быть от 59% до 89%. Такой разброс показывают модели как с применением over-sampling (ROS), так и без. **В обоих случаях простое среднее всех 15 результатов - 74-75%**. Это соответствует наиболее частому результату, который мы получали ранее, комбинируя разные гиперпараметры и методы при обучении.

## Общие итоги
Добиться стойкого увеличения качества предсказания за счет борьбы с дисбалансом классов и настройки гиперпараметров не получается. Хотя результаты варьируется довольно сильно (как видно из отработки kfold - в приделах 30%!), достижение наилучших показателей (самый высокий - 89%) на практике происходит случайно: в процессе многократного перезапуска одной и той же комбинации векторизатора, сплита и классификатора, а не за счет контролируемого изменения отдельных настроек. 

По всей видимости, на имеющихся данных качество обучения/предсказания зависит от того, как случайным образом разбились данные на обучающую и тестовую выборку (условно говоря, насколько модели "повезло на экзамене"). А повышать качество обучения для достижения стабильных результатов нужно, вероятно, за счет работы с самими данными: 
1. Обогощать данными миноритарные классы
2. Возможно - пересматривать экстралингвистический подход к категоризации
3. Возможно - искусственно делить тексты на несколько текстов, т.к. они сравнительно объемные (больше короткой новостной заметки или твита)